## Imports

In [1]:
import os
from pathlib import Path
import gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

## Load Environment

In [2]:
# load the env from gym
env_name = "CartPole-v0"
env = gym.make(env_name)

In [3]:
# test env by using random valid actions from the action space
nb_episodes = 5
for ep in range(1, nb_episodes+1):
    state = env.reset()
    done = False
    score = 0

    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score += reward
        
    print(f"episode: {ep},\tscore:{score}")


episode: 1,	score:29.0
episode: 2,	score:17.0
episode: 3,	score:30.0
episode: 4,	score:32.0
episode: 5,	score:12.0


In [4]:
env.close()

## Train Model

In [5]:
# prepare tensorboard log
log_path = os.path.join("..", "training", "logs")
Path(log_path).mkdir(parents=True, exist_ok=True)

In [6]:
# create env and agent
env = gym.make(env_name)#, render_mode=None)
env = DummyVecEnv([lambda: env])
model = PPO(policy="MlpPolicy", env=env, verbose=1, tensorboard_log=log_path)

Using cpu device


In [7]:
# train
model.learn(total_timesteps=20000)

Logging to ..\training\logs\PPO_2
-----------------------------
| time/              |      |
|    fps             | 1992 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 1250         |
|    iterations           | 2            |
|    time_elapsed         | 3            |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0030965102 |
|    clip_fraction        | 0.0825       |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.687       |
|    explained_variance   | 0.00533      |
|    learning_rate        | 0.0003       |
|    loss                 | 5.54         |
|    n_updates            | 10           |
|    policy_gradient_loss | -0.0131      |
|    value_loss           | 56.3         |
-------------------------

## Save and Reload Model

In [8]:
# prepare save location
save_models_path = os.path.join("..", "training", "saved_models")
Path(save_models_path).mkdir(parents=True, exist_ok=True)
ppo_save_path = os.path.join(save_models_path, "ppo_cart_pole")

In [9]:
# save model
model.save(ppo_save_path)

In [10]:
# remove model from memory
del model

In [11]:
# reload model from saved files
model = PPO.load(ppo_save_path, env=env)

## Evaluate Policy

In [12]:
score_avg, score_std = evaluate_policy(model, env, n_eval_episodes=10, render=True)
env.close()

c:\Users\preto\anaconda3\envs\cart-pole\lib\site-packages\stable_baselines3\common\evaluation.py:65: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


In [13]:
print(f"Score (avg): {score_avg}")
print(f"Score (std): {score_std}")

Score (avg): 200.0
Score (std): 0.0


## Test Model

In [14]:
# test trained model actions in env
nb_episodes = 5
for ep in range(1, nb_episodes+1):
    obs = env.reset()
    done = False
    score = 0

    while not done:
        env.render()
        action, _ = model.predict(obs)
        obs, reward, done, info = env.step(action)
        score += reward
        
    print(f"episode: {ep},\tscore:{score}")


episode: 1,	score:[200.]
episode: 2,	score:[200.]
episode: 3,	score:[200.]
episode: 4,	score:[200.]
episode: 5,	score:[200.]


In [15]:
env.close()

## Tensorboard Logs

Run the following from the terminal:
`tensorboard --logdir=<log-path>`

See the below cell to get the log path

In [21]:
print(f"tensorboard --logdir=\"{log_path}\"")

tensorboard --logdir="..\training\logs"
